# Prediccion de demanda

In [187]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## 1. Con reglas duras  de las variaciones por dia

**Plan:**
1. Sacar entre dias del anios consecutivos la variacion
2. Promediar (o con mediana) la variacion media al dia
3. Multiplicar los valores de los ultimos dias por la variacion calculada en el punto 2.

In [188]:
demanda_df = pd.read_csv(".\Datos\demanda.csv")

<>:1: SyntaxWarning: invalid escape sequence '\D'
<>:1: SyntaxWarning: invalid escape sequence '\D'
C:\Users\juanc\AppData\Local\Temp\ipykernel_25612\1703249404.py:1: SyntaxWarning: invalid escape sequence '\D'
  demanda_df = pd.read_csv(".\Datos\demanda.csv")


In [189]:
demanda_prod_df = demanda_df[demanda_df['id_producto']==1]

In [190]:
demanda_prod_df.head()

,date,id_producto,demanda
127,2020-01-02,1,110
253,2020-01-03,1,140
380,2020-01-04,1,121
508,2020-01-05,1,140
766,2020-01-07,1,102


In [191]:
demanda_prod_df['date'] = pd.to_datetime(demanda_prod_df['date'], format='%Y-%m-%d')
demanda_prod_df['anio'] = demanda_prod_df['date'].dt.year
demanda_prod_df['mes'] = demanda_prod_df['date'].dt.month
demanda_prod_df['dia'] = demanda_prod_df['date'].dt.day

C:\Users\juanc\AppData\Local\Temp\ipykernel_25612\56672937.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  demanda_prod_df['date'] = pd.to_datetime(demanda_prod_df['date'], format='%Y-%m-%d')
C:\Users\juanc\AppData\Local\Temp\ipykernel_25612\56672937.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  demanda_prod_df['anio'] = demanda_prod_df['date'].dt.year
C:\Users\juanc\AppData\Local\Temp\ipykernel_25612\56672937.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a D

In [192]:
demanda_prod_df.head()

,date,id_producto,demanda,anio,mes,dia
127,2020-01-02,1,110,2020,1,2
253,2020-01-03,1,140,2020,1,3
380,2020-01-04,1,121,2020,1,4
508,2020-01-05,1,140,2020,1,5
766,2020-01-07,1,102,2020,1,7


In [193]:
# Crear un dataframe vacio
demanda_prod_df = demanda_prod_df.sort_values(by='date', ascending=True)
demanda_prod_df.reset_index(drop=True, inplace=True)

# Crear una columna indicadora para el 29 de febrero
demanda_prod_df['is_feb_29'] = demanda_prod_df['date'].apply(lambda x: x.month == 2 and x.day == 29)
demanda_prod_df = demanda_prod_df[~demanda_prod_df['is_feb_29']]

# Crear la clave mes-día
demanda_prod_df['mes_dia'] = demanda_prod_df['mes'].astype(str) + '-' + demanda_prod_df['dia'].astype(str)

# Crear la tabla pivote
transformed_df = demanda_prod_df.pivot_table(index='mes_dia', columns='anio', values='demanda')

# Resetear el índice y eliminar nombres de índice no deseados
transformed_df.reset_index(inplace=True)
transformed_df.columns.name = None  # Quita el nombre del índice de columnas

In [194]:
# Renombrar columnas de años a 'demanda_{anio}'
transformed_df = transformed_df.rename(columns=lambda x: f"demanda_{x}" if isinstance(x, int) else x)

In [195]:
# Calculo de diferencia de datos
cols_demanda = [c for c in transformed_df.columns if 'demanda' in c]
for i,c in enumerate(cols_demanda):
    if i>0:
        transformed_df['pct_var_rel_{a}_{am1}'.format(a=cols_demanda[i][-4:], am1=cols_demanda[i-1][-4:])] = \
            (transformed_df[cols_demanda[i]] - transformed_df[cols_demanda[i-1]]) / transformed_df[cols_demanda[i-1]]

In [ ]:
# Calcular la media del porcentage de varianza relativo

,mes_dia,demanda_2020,demanda_2021,demanda_2022,pct_var_rel_2021_2020,pct_var_rel_2022_2021
0,1-1,NaN,183.0,134.0,NaN,-0.267760
1,1-10,115.0,120.0,83.0,0.043478,-0.308333
2,1-11,95.0,139.0,74.0,0.463158,-0.467626
3,1-12,127.0,102.0,76.0,-0.196850,-0.254902
4,1-13,88.0,107.0,62.0,0.215909,-0.420561
...,...,...,...,...,...,...
360,9-5,71.0,67.0,43.0,-0.056338,-0.358209
361,9-6,79.0,40.0,45.0,-0.493671,0.125000
362,9-7,64.0,51.0,48.0,-0.203125,-0.058824
363,9-8,63.0,48.0,45.0,-0.238095,-0.062500


# 2. Con un modelo Autoregresivo

# 3. Con una Red Neuronal Recurrente o LSTM